# Notebook 0 : Test de Connexion aux Services GCP

## Objectif

Ce notebook permet de **vérifier la configuration et la connexion** aux services Google Cloud Platform (GCP) nécessaires pour le projet ETL :
- **Google Cloud Storage (GCS)** : pour le stockage des données brutes (couche "bronze")
- **BigQuery** : pour le traitement et le stockage des données transformées (couche "silver")

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :

1. **Fichier `.env` configuré** à la racine du projet avec :
   - `PROJECT_ID` : l'identifiant de votre projet GCP
   - `GOOGLE_APPLICATION_CREDENTIALS` : le chemin vers votre fichier de credentials JSON
   - `BUCKET_NAME` : le nom de votre bucket GCS

2. **Service Account** avec les permissions suivantes :
   - `Storage Object Viewer` ou `Storage Admin` pour GCS
   - `BigQuery Data Viewer` et `BigQuery Job User` pour BigQuery

3. **Packages Python installés** :
   ```bash
   pip install -r requirements.txt



In [5]:
import os
from dotenv import load_dotenv
from pathlib import Path
from google.oauth2 import service_account
from google.cloud import storage, bigquery

load_dotenv()

ROOT = Path.cwd().parent

# Id du projet
PROJECT_ID  = os.getenv("PROJECT_ID")
# Configuration du service account
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
# Configuration du bucket GCS
BUCKET_NAME = os.getenv("BUCKET_NAME")

## 1 - Configuration
- Chargement des variables d'environnement depuis `.env`
- Initialisation des chemins et identifiants du projet


In [6]:
# 1) Charger les credentials du fichier JSON
creds = service_account.Credentials.from_service_account_file(SA_PATH)

## 2 - Test de Connexion GCS
- Authentification avec le Service Account
- Vérification de l'accès au projet
- Listing des buckets disponibles
- **Inspection du bucket** : affichage de tous les fichiers stockés dans le bucket configuré


In [7]:
# 2) Créer le client GCS
st = storage.Client(project=PROJECT_ID, credentials=creds)

print(f"Auth OK - storage : {st.project}")
print(f"Liste des buckets : {list(st.list_buckets())}")

# Lister les fichiers dans le bucket
bucket = st.bucket(BUCKET_NAME)
print(f"Fichiers dans le bucket '{BUCKET_NAME}':")
print("-" * 50)

blobs = bucket.list_blobs()
file_count = 0

for blob in blobs:
    print(f"[FILE] {blob.name}")
    file_count += 1

print(f"\nTotal: {file_count} fichiers trouvés")

Auth OK - storage : univ-reims-sncf-forecast
Liste des buckets : [<Bucket: bronze-sncf-etl>]
Fichiers dans le bucket 'bronze-sncf-etl':
--------------------------------------------------
[FILE] bronze/arrets/arrets.parquet
[FILE] bronze/emplacement-des-gares-idf/emplacement-des-gares-idf.parquet
[FILE] bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S1_NB_FER.csv
[FILE] bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S1_PROFIL_FER.csv
[FILE] bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S2_NB_FER.csv
[FILE] bronze/histo-validations-reseau-ferre/2015/data-rf-2015/2015S2_PROFIL_FER.csv
[FILE] bronze/histo-validations-reseau-ferre/2016/data-rf-2016/2016S1_NB_FER.txt
[FILE] bronze/histo-validations-reseau-ferre/2016/data-rf-2016/2016S1_PROFIL_FER.txt
[FILE] bronze/histo-validations-reseau-ferre/2016/data-rf-2016/2016S2_NB_FER.txt
[FILE] bronze/histo-validations-reseau-ferre/2016/data-rf-2016/2016S2_PROFIL_FER.txt
[FILE] bronze/histo-validations-reseau-

## 3) - Test de Connexion BigQuery
- Création du client BigQuery
- **Vérification du projet** : confirmation de l'accès au projet
- **Listing des datasets** : affichage de tous les datasets disponibles
- **Test d'exécution SQL** : exécution d'une requête simple pour valider les permissions

In [8]:
# 3) Créer le client BigQuery
bq = bigquery.Client(project=PROJECT_ID, credentials=creds)

# Test 1: Vérifier le projet
print(f"\n[OK] - Projet BigQuery: {bq.project}")

# Test 2: Lister les datasets
print(f"\n[...] - Liste des datasets dans le projet:")
datasets = list(bq.list_datasets())
if datasets:
    for dataset in datasets:
        print(f"  - {dataset.dataset_id}")
else:
    print("  (Aucun dataset trouvé)")

# Test 3: Exécuter une requête simple
print(f"\n[...] - Test d'exécution d'une requête SQL...")
query = "SELECT 1 as test_value, CURRENT_TIMESTAMP() as timestamp"
results = bq.query(query).result()
for row in results:
    print(f"    [OK] - Requête exécutée avec succès")
    print(f"         - Test value: {row.test_value}")
    print(f"         - Timestamp: {row.timestamp}")
    



[OK] - Projet BigQuery: univ-reims-sncf-forecast

[...] - Liste des datasets dans le projet:
  - silver

[...] - Test d'exécution d'une requête SQL...
    [OK] - Requête exécutée avec succès
         - Test value: 1
         - Timestamp: 2025-11-17 20:37:26.565341+00:00
